In [0]:
df=spark.read.parquet("/mnt/bronze/npi_extract")
df.createOrReplaceTempView('npi_extract')

In [0]:

%sql
CREATE TABLE IF NOT EXISTS `hrcm-data-catalog`.silver.npi_extract (
  npi_id STRING,
  first_name STRING,
  last_name STRING,
  position STRING,
  organisation_name STRING,
  last_updated STRING,
  inserted_date DATE,
  updated_date DATE,
  is_current_flag BOOLEAN
)

In [0]:
%sql
MERGE INTO
  `hrcm-data-catalog`.silver.npi_extract AS target
USING
  npi_extract AS source
ON target.npi_id = source.npi_id and target.is_current_flag = true
WHEN MATCHED AND
  target.first_name != source.first_name OR
  target.last_name != source.last_name OR
  target.position != source.position OR
  target.organisation_name != source.organisation_name OR
  target.last_updated != source.last_updated
  THEN UPDATE SET
  target.updated_date = current_date,
  target.is_current_flag = False


In [0]:

%sql
MERGE INTO
  `hrcm-data-catalog`.silver.npi_extract AS target
USING
  npi_extract AS source
ON target.npi_id = source.npi_id and target.is_current_flag = true
WHEN NOT MATCHED THEN INSERT (
   npi_id,
  first_name ,
  last_name ,
  position ,
  organisation_name ,
  last_updated ,
  inserted_date ,
  updated_date ,
  is_current_flag 
)
  VALUES (
    source.npi_id,
  source.first_name ,
  source.last_name ,
  source.position ,
  source.organisation_name ,
  source.last_updated ,
  current_date,
  current_date, 
  true
  )

In [0]:
df_silver = spark.sql("select * from `hrcm-data-catalog`.silver.npi_extract")
df_silver.write.format("delta").mode("overwrite").save("/mnt/silver/npi_extract")